<a href="https://colab.research.google.com/github/jeffvestal/elastic_jupyter_notebooks/blob/main/generate_vectors_on_ingest_to_elastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Vectors on ingest into Elasticsearch

This code will show you how to load a supported embedding model from Hugging Face into an elasticsearch cluster in [Elastic Cloud](https://cloud.elastic.co/)

First we will configure our python environment

1. Set up our python environment
2. Load the chosed model from HF into Elasticsearch
3. Deploy and start the model on ML nodes


### Elastic version support
Requires Elastic version 8.0+ with a platinum or enterprise license (or trial license)

You can set up a [free trial elasticsearch Deployment in Elastic Cloud](https://cloud.elastic.co/registration).

# Setup
This section will set up the python environment with the required libraries

## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [ ]:
pip install eland

In [ ]:
pip install elasticsearch

In [ ]:
pip install transformers

In [ ]:
pip install sentence_transformers

In [ ]:
pip install torch==1.11

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

## Configure elasticsearch authentication. 
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [ ]:
import getpass

In [ ]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_id = getpass.getpass('Enter cluster API key ID:  ') 
es_api_key = getpass.getpass('Enter cluster API key:  ')

## Connect to Elastic Cloud

In [ ]:
es = Elasticsearch(cloud_id=es_cloud_id, 
                   api_key=(es_api_id, es_api_key)
                   )
es.info() # should return cluster info

# Loading a model from Hugging Face model hub

The Elastic Stack machine learning features support transformer models that conform to the standard BERT model interface and use the WordPiece tokenization algorithm.

A current list of supported architectures can be viewed in the
[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html)


## Download an embedding model from Hugging Face using the HF copy link

For this example we will be using the [sentence-transformers/msmarco-MiniLM-L-12-v3](https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-12-v3) model


### Download the Model
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page. 

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code: 
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [ ]:
hf_model_id='sentence-transformers/msmarco-MiniLM-L-12-v3'
tm = TransformerModel(hf_model_id, "text_embedding")

### Set and confirm the model ID
To make the name compatible with elasticsearch, the '/' is replaced with '__'



In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

## Export the model in a TorchScrpt representation which Elasticsearch uses

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

## Load the model into Elasticsearch
Model should not already exist in elasticsearch

In [ ]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

# Starting the Model

## View information about the model
This is not required but can be handy to get a model overivew

In [ ]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

## Verify the model started without issue

In [ ]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

---
---
# Working with Vectors
---



# Setting up the index in elasticsearch to store vectors



## Generate Vector for Query

Before we can run a kNN query, we need to convert our query string to a vector.

Creating a sample query sentence

In [ ]:
docs =  [
    {
      "text_field": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
    }
  ]

We call the `_infer` endpoint supplying the model_id and the doc[s] we want to vectorize. 

In [ ]:
z = MlClient.infer_trained_model(es, model_id=es_model_id, docs=docs, )

The vector for the first doc can be accessed in the response dict as shown below

In [ ]:
doc_0_vector = z['inference_results'][0]['predicted_value']
doc_0_vector